In [37]:
#Import stuff
import numpy as np
from scipy import fftpack as fft
from scipy.io import wavfile as wav
import os as system
import csv 

command_file = "input.csv"

#calculate IR for single sample
def sample_IR(sample_mem, IR_pos, IR):
    '''calculate single sample of IR filter from memory of past samples and IR
    sample mem and IR should be the same length
    pos is the start position for the samples'''
    sample = 0.0
    for x in range(0, len(sample_mem)):
        sample += sample_mem[(IR_pos+x)%len(sample_mem)]*IR[len(IR)-x-1]
        
    return sample
    
#string sim function
def step_string(string, pos, length, last_length, reset, pluck, IR_mem, IR_pos, filter_IR):
    '''function for incrementing the string simulation by 1 step
    returns the sample for that step of simulation
    pos will be incremented after each step
    IR_pos will also be incremented''' 
    
    if (length > last_length):
        if (((pos)%len(string)) > ((pos+length)%len(string))):
            string[int((pos+length)%len(string)):int((pos)%len(string))] = 0
        else:
            string[0:int((pos)%len(string))] = 0
            string[int((pos+length)%len(string)):int(len(string))] = 0
            
    if reset:#reset string
        string = pluck
        for x in range(0, len(string)):
            string[int((pos+x)%len(string))] = pluck[x]
        return 0, string, IR_mem, pos, IR_pos, length
       
    else:
        sample = string[pos%len(string)]
        IR_mem[IR_pos%len(IR_mem)] = sample
        string[int((pos+length-1)%len(string))] = sample_IR(IR_mem, IR_pos, filter_IR)
        return sample, string, IR_mem, pos+1, IR_pos+1, length

#make string from given parameters
def make_string(sample_rate, min_f, oversampling, filter_IR):
    '''create string'''
    IR_mem = np.zeros(len(filter_IR))
    string = np.zeros(sample_rate*min_f*oversampling)
    return string, IR_mem, 0, 0

#make IR for lowpass filter
def make_lowpass_IR(sample_rate, oversampling, f_cutoff, gain):
    '''create lowpass IR to be used for the string
    gain is the gain for every cycle. around 1 should be sustained signal'''
    filter_IR = np.zeros(int((sample_rate*oversampling)/(f_cutoff*2)))
    filter_IR[0:len(filter_IR) - 1] = (gain)/(len(filter_IR))
    return filter_IR

#get length of the string to use
def get_length(sample_rate, oversampling, frequency, lenIR):
    '''returns length of string to use'''
    return (sample_rate*oversampling)/(frequency) - lenIR/2

#make the pluck shape
def make_pluck(string_length, pluck_length, magnitude):
    '''create the pluck chape to be copied'''
    
    pluck = np.zeros(string_length)
    pluck[0:int(pluck_length)+1] = np.arange(-1,1,(2/(pluck_length)))
    #pluck[0:pluck_length/4)+1] = np.arange(-1,1,(2/(pluck_length/4)))
    #pluck[int(pluck_length/4):pluck_length+1] = np.arange(1,-1,-(2/(pluck_length*3/4)))
    
    return pluck*magnitude

#generate the whole string simulation for 1 string
def string_sim(pluck_time, pluck_amp, length_time, Length_freq, damp_time, damp_fac, filter_IR_raw, length, sr, os, min_f, sn):   
    '''runs the string sim for the whole time'''
    
    samples = np.zeros(int(length/os))
    t = 0#current time in number of ticks/steps for the string sim
    
    string, IR_mem, pos, IR_pos = make_string(sr, min_f, os, filter_IR_raw)
    last_length = 0
    
    length_pos = 0
    f = 110
    
    damp_pos = 0
    damp = 1
    filter_IR = filter_IR_raw*damp
    
    pluck_pos = 0
    reset = 0
    pluck_strength = 1
    pluck = make_pluck(len(string), get_length(sr, os, f, len(filter_IR)), pluck_strength)
    
    lp = 0;
    
    for x in range(0, len(samples)):
        sample_sum = 0.0
        for y in range(0, os):
                
            if ((damp_pos < len(damp_time)) and (damp_time[damp_pos] <= t)):
                damp = damp_fac[damp_pos]
                filter_IR = filter_IR_raw*damp
                damp_pos += 1
                
            if ((length_pos < len(length_time)) and (length_time[length_pos] <= t)):
                f = Length_freq[length_pos]
                length_pos += 1
                
            if ((pluck_pos < len(pluck_time)) and (pluck_time[pluck_pos] <= t)):
                reset = 1
                pluck_strength = pluck_amp[pluck_pos]
                pluck = make_pluck(len(string), get_length(sr, os, f, len(filter_IR)), pluck_strength)
                pluck_pos += 1
            else:
                reset = 0

            sample_a, string, IR_mem, pos, IR_pos, last_length = step_string(string, pos, get_length(sr, os, f, len(filter_IR)), last_length, reset, pluck, IR_mem, IR_pos, filter_IR)
            sample_sum += sample_a
            t += 1
        
        samples[x] = (sample_sum)/os #oversample the string simulation
        
        
        if(int(t*20/length) > lp):
            print(str(int(t*20/length)*5) + '% done on string ' + str(sn)) #print progress
            lp = int(t*20/length)
            
    return samples

sr = 96000
os = 2
min_f = 20


length = 300000;#length of sound in samples
print("starting input parsing")

'''
pluck_time0 = np.empty() #create empty arrays for 
pluck_time1 = np.empty()
pluck_time2 = np.empty()
pluck_time3 = np.empty()
pluck_time4 = np.empty()
pluck_time5 = np.empty()

pluck_time0 = np.empty() 
pluck_time1 = np.empty()
pluck_time2 = np.empty()
pluck_time3 = np.empty()
pluck_time4 = np.empty()
pluck_time5 = np.empty()

pluck_time0 = np.empty() 
pluck_time1 = np.empty()
pluck_time2 = np.empty()
pluck_time3 = np.empty()
pluck_time4 = np.empty()
pluck_time5 = np.empty()
'''


print("starting sample generation")
#samples = string_sim(pluck_time, pluck_amp, length_time, Length_freq, damp_time, damp_fac, filter_IR_raw, length*os, sr, os, min_f, 0)

samples = string_sim(np.array([0, 100000*os, 150000*os]), np.array([1, 0.2, 0.7]), np.array([0, 150000*os]), np.array([110, 196]), np. array([0, 250000*os]), np.array([1, 0.9]), make_lowpass_IR(sr, os, 3000, 1.03), length*os, sr, os, min_f, 0)



'''
samples0 = string_sim(pluck_time0, pluck_amp0, length_time0, Length_freq0, damp_time0, damp_fac0, make_lowpass_IR(sr, os, 3000, 1.03), length*os, sr, os, min_f, 0)
samples1 = string_sim(pluck_time1, pluck_amp1, length_time1, Length_freq1, damp_time1, damp_fac1, make_lowpass_IR(sr, os, 3000, 1.03), length*os, sr, os, min_f, 1)
samples2 = string_sim(pluck_time2, pluck_amp2, length_time2, Length_freq2, damp_time2, damp_fac2, make_lowpass_IR(sr, os, 3000, 1.03), length*os, sr, os, min_f, 2)
samples3 = string_sim(pluck_time3, pluck_amp3, length_time3, Length_freq3, damp_time3, damp_fac3, make_lowpass_IR(sr, os, 3000, 1.03), length*os, sr, os, min_f, 3)
samples4 = string_sim(pluck_time4, pluck_amp4, length_time4, Length_freq4, damp_time4, damp_fac4, make_lowpass_IR(sr, os, 3000, 1.03), length*os, sr, os, min_f, 4)
samples5 = string_sim(pluck_time5, pluck_amp5, length_time5, Length_freq5, damp_time5, damp_fac5, make_lowpass_IR(sr, os, 3000, 1.03), length*os, sr, os, min_f, 5)
samples = samples0 + samples1 + samples2 + samples3 + samples4 + samples5
'''

    
print("applying reverb")

fs, guitar_IR = wav.read('Guitar IR EQ Edited.wav')
fs, room_L_IR = wav.read('Room IR Left Very Edited.wav')
fs, room_R_IR = wav.read('Room IR Right Very Edited.wav')
samples_fft = fft.fft(np.concatenate([samples, np.zeros(length - len(samples))]))
guitar_IR_fft = fft.fft(np.concatenate([guitar_IR/sum(guitar_IR), np.zeros(length - len(guitar_IR))]))
room_L_IR_fft = fft.fft(np.concatenate([room_L_IR/sum(room_L_IR), np.zeros(length - len(room_L_IR))]))
room_R_IR_fft = fft.fft(np.concatenate([room_R_IR/sum(room_R_IR), np.zeros(length - len(room_R_IR))]))

result_L_fft = samples_fft*guitar_IR_fft*room_L_IR_fft
result_R_fft = samples_fft*guitar_IR_fft*room_R_IR_fft
    
result_L = fft.ifft(result_L_fft)
result_R = fft.ifft(result_R_fft)

result_L = result_L/np.amax(result_L)  #normalise each channel
result_R = result_R/np.amax(result_R)
    
print("writing output")
wav.write("result.wav", sr, np.array([result_L.astype('float'), result_R.astype('float')], np.float).T)#synth output 
wav.write("string.wav", sr, samples.astype('float'))#string sim output for testing
print("converting output wav to opus")
system.system("ffmpeg -i result.wav -c:a libopus -b:a 192k result.opus")#convert output to something more reasonable in size
system.system("ffmpeg -i string.wav -c:a libopus -b:a 192k string.opus")
print("converting output wav to mp3")
system.system("ffmpeg -i result.wav -c:a libmp3lame -q:a 2 result.mp3")#convert output to something easier to share
system.system("ffmpeg -i string.wav -c:a libmp3lame -q:a 2 string.mp3")
print("done")

starting sample generation
5% done on string 0
10% done on string 0
15% done on string 0
20% done on string 0
25% done on string 0
30% done on string 0
35% done on string 0
40% done on string 0
45% done on string 0
50% done on string 0
55% done on string 0
60% done on string 0
65% done on string 0
70% done on string 0
75% done on string 0
80% done on string 0
85% done on string 0
90% done on string 0
95% done on string 0
100% done on string 0
applying reverb
writing output
converting output wav to opus


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:187: ComplexWarning: Casting complex values to real discards the imaginary part


converting output wav to mp3
done
